In [2]:
from ann_automl.utils.process import process, set_pstate, get_pstate_value, request, pstate
from ann_automl.jupyter.processvis import show_progress


@process
def f(n):
    set_pstate(status="Step 1")
    for i in range(n):
        if i%100000 == 0:
            set_pstate(state=f'{i+1} of {n}', progress=(i+1)/n)
            if pstate()._stop:
                return
    set_pstate(status="Wait for input integer")
    print("call example")
    a = request('input_int')
    set_pstate(status="Step 2")
    for i in range(a):
        if i%100000 == 0:
            set_pstate(state=f'{i+1} of {n}', progress=(i+1)/n)
            if pstate()._stop:
                return
    return a

p = f(100000000, start=False)
show_progress(p)

Label(value='abcd', layout=Layout(width='100%'))

In [4]:
from ann_automl.utils.pipe import pipe, send, receive
from ann_automl.jupyter.pipevis import show_progress


@pipe
def f(n):
    from ann_automl.utils.pipe import send, receive
    send("status", "Step 1")
    for i in range(n):
        if i%100000 == 0:
            send("set", state=f'{i+1} of {n}', progress=(i+1)/n)
    send("status", "Wait for input integer")
    print("call example")
    a = receive('input_int')
    send("set", status=f"Step 2 : a={a}")
    for i in range(a):
        if i%100000 == 0:
            send("set", state=f'{i+1} of {n}', progress=(i+1)/n)
    return a

p = f(100000000, start=False)
show_progress(p)

Label(value='abcd', layout=Layout(width='100%'))

call example


TypeError: 'NoneType' object is not callable

In [ ]:
def f(n):
    for i in range(n):
        if i%100000 == 0:
            set_pstate(state=f'{i+1} of {n}', progress=(i+1)/n)
            if pstate()._stop:
                return

@process
def g(n):
    set_pstate(status="Step 1")
    f(n)
    set_pstate(status="Wait for input integer")
    a = request('input_int')
    set_pstate(status="Step 2")
    f(a)
    return n

p = g(100000000, start=False)
show_progress(p)

In [ ]:
##############################################

## Тест графического интерфейса всей системы

In [5]:
from ann_automl.core.solver import Task, solve
from ann_automl.core import nn_rules
from ann_automl.utils.process import process
import ipywidgets
import traceback
import sys

class SolverGUI:
    def create_main_screen(self):
        style = {'description_width': '200px', 'width' : '500px'}
        ct_in = ipywidgets.Dropdown(
            options=['train', 'test', 'database', 'history'],
            value='train',
            description='Категория задачи:',
            style = style
        )

        type_in = ipywidgets.Dropdown(
            options=['classification', 'segmentation', 'detection'],
            value='classification',
            description='Тип задачи:',
            style = style
        )

        cat_in = ipywidgets.Text(
            value='cats, dogs',
            description='Категория объектов интереса:',
            style = style
        )

        target_in = ipywidgets.Dropdown(
            options=['loss', 'metrics'],
            value='metrics',
            description='Цель:',
            style = style
        )

        target_val_in = ipywidgets.BoundedFloatText(
            value=0.7,
            min=0,
            max=1,
            step=0.05,
            description='Значение цели:',
            style = style
        )
    
        def on_finish(p):
            start.disabled = False
            if isinstance(p._exn, Exception):
                try:
                    raise p._exn
                except Exception as e:
                    self.handlers["log"](f'Process finished with error:')
                    self.handlers["error"](traceback.format_exc())
            else:
                self.handlers["log"](f'Process finished')

        def on_start_clicked(b):
            self._log_widget.clear_output()
            with self._log_widget:
                print("========= Process output ==========")
            obj = [x.strip() for x in str(cat_in.value).split(",")]
            task = Task(ct_in.value, task_type=type_in.value, obj_set=obj, goal={target_in.value: target_val_in.value})
            self.proc = process(solve)(task, debug_mode=True, start=False)
            self.proc.handlers.update(self.handlers)
            self.proc.start()
            self.proc.on_finish = on_finish
            start.disabled = True
            
                

        start = ipywidgets.Button(description='Запустить систему',
                                  #style = {'button_color': 'rgb(0,140,255)'},
                                  layout=ipywidgets.Layout(width='500px', height='28px'))
        start.on_click(on_start_clicked)

        return ipywidgets.VBox([ct_in, type_in, cat_in, target_in, target_val_in, start])

    def create_strategy_found(self):
        a11 = ipywidgets.Dropdown(
            options=[('Стратегия 1', 1), ('Стратегия 2', 2), ('Стратегия 3', 3)],
            value=1,
            description='Найдены подходящие стратегии:',
            style = {'description_width': '200px', 'width' : '600px'}
        )

        def on_button_clicked7(b):
            print('Список 1:', b['old'], '->', b['new'])

        btn_yes = ipywidgets.Button(description='Да', style = {'button_color': 'rgb(0,255,0)'})
        btn_no = ipywidgets.Button(description='Нет', style = {'button_color': 'rgb(255,0,0)'})

        def on_btn_yes(b):
            self.tab.selected_index = 4
        def on_btn_no(b):
            self.tab.selected_index = 0

        btn_yes.on_click(on_btn_yes)
        btn_no.on_click(on_btn_no)

        HB = ipywidgets.HBox([ipywidgets.Label('Хотите обучить новую модель или закончить работу? : '), btn_yes, btn_no])
        return ipywidgets.VBox([a11, HB])

    def create_retrain_widget(self):
        style = {'button_color': 'rgb(0,140,255)'}
        c1 = ipywidgets.Button(
            description='Выбрать другую стратегию (из истории системы)',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            layout=ipywidgets.Layout(width='500px', height='28px'),
            style = style
        )
        c2 = ipywidgets.Button(
            description='Вручную задать параметры обучения',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            layout=ipywidgets.Layout(width='500px', height='28px'),
            style = style
        )
        c3 = ipywidgets.Button(
            description='Обучение по сетке',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            layout=ipywidgets.Layout(width='500px', height='28px'),
            style = style
        )
        c4 = ipywidgets.Button(
            description='Изменить архитектуру модели (случайно\по истории\по спец. технологии)',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            layout=ipywidgets.Layout(width='500px', height='28px'),
            style = style
        )
        c5 = ipywidgets.Button(
            description='изменить расписание скорости обучения',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            layout=ipywidgets.Layout(width='500px', height='28px'),
            style = style
        )
        d1 = ipywidgets.VBox([ipywidgets.Label('Приёмы:'), c1, c2, c3, c4, c5])
        e1 = ipywidgets.FloatText(
            value=0.5,
            disabled=False,
            layout=ipywidgets.Layout(width='80px', height='28px')
        )
        e2 = ipywidgets.FloatText(
            value=0.5,
            disabled=False,
            layout=ipywidgets.Layout(width='80px', height='28px')
        )
        e3 = ipywidgets.FloatText(
            value=0.5,
            disabled=False,
            layout=ipywidgets.Layout(width='80px', height='28px')
        )
        e4 = ipywidgets.FloatText(
            value=0.5,
            disabled=False,
            layout=ipywidgets.Layout(width='80px', height='28px')
        )
        e5 = ipywidgets.FloatText(
            value=0.5,
            disabled=False,
            layout=ipywidgets.Layout(width='80px', height='28px')
        )
        d2 = ipywidgets.VBox([ipywidgets.Label('Текущее распределение вероятности «успеха»'), e1, e2, e3, e4, e5])
        d3 = ipywidgets.HBox([d1, d2])
        c6 = ipywidgets.Button(
            description='Случайный прием',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            layout=ipywidgets.Layout(width='200px', height='50px'),
            style = {'button_color': 'rgb(204,0,255)'}
        )
        c7 = ipywidgets.Button(
            description='Закончить обучение',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            layout=ipywidgets.Layout(width='200px', height='50px'),
            style = {'button_color': 'rgb(255,119,0)'}
        )
        d4 = ipywidgets.HBox([c6, c7])
        d5 = ipywidgets.VBox([d3, d4])
        return d5

    def create_manual_params(self):
        a7 = ipywidgets.Dropdown(
            options=[('classification', 1), ('segmentation', 2), ('detection', 3)],
            value=2,
            description='Параметры базы данных:',
            style = {'description_width': '200px', 'width' : '500px'}
        )

        def on_button_clicked7(b):
            print('Список 1:', b['old'], '->', b['new'])

        a8= ipywidgets.Dropdown(
            options=[('classification', 1), ('segmentation', 2), ('detection', 3)],
            value=2,
            description='Параметры обучения:',
            style = {'description_width': '200px', 'width' : '500px'}
        )

        def on_button_clicked8(b):
            print('Список 2:', b['old'], '->', b['new'])

        a9 = ipywidgets.Dropdown(
            options=[('classification', 1), ('segmentation', 2), ('detection', 3)],
            value=2,
            description='Архитектура модели:',
            style = {'description_width': '200px', 'width' : '500px'}
        )

        def on_button_clicked9(b):
            print('Список 3:', b['old'], '->', b['new'])

        a10 = ipywidgets.Button(
            description='Прекратить обучение',
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            layout=ipywidgets.Layout(width='200px', height='40px'),
            style = {'button_color': 'rgb(0,178,255)'}
        )

        a7.observe(on_button_clicked7, names='value')
        a8.observe(on_button_clicked8, names='value')
        a9.observe(on_button_clicked9, names='value')

        return ipywidgets.VBox([a7, a8, a9, a10])

    def create_train_choose(self):
        h1 = ipywidgets.Button(
            description='Вручную задать параметры обучения',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            layout=ipywidgets.Layout(width='640px', height='80px'),
            style = {'button_color': 'rgb(255,255,0)'}
        )
        h2 = ipywidgets.Button(
            description='Обучить одну модель с т.зр. лучшей стратегии по сохраненной в системе истории обучения моделей',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            layout=ipywidgets.Layout(width='640px', height='80px'),
            style = {'button_color': 'rgb(255,255,0)'}
        )
        h3 = ipywidgets.Button(
            description='Обучение по сетке',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            layout=ipywidgets.Layout(width='640px', height='80px'),
            style = {'button_color': 'rgb(255,255,0)'}
        )
        def on_button_clicked1(b): # Описываем обработчик события
            self.tab.selected_index = 2
        h1.on_click(on_button_clicked1)
        return ipywidgets.VBox( [h1, h2, h3])

    def __init__(self, ptask):
        self._log = []  # логи
        self._ptask = ptask
        self.handlers = {}
        self.btn_next = ipywidgets.Button(description='next')
        self.btn_prev = ipywidgets.Button(description='prev')
        def on_next(b):
            self.tab.selected_index = self.tab.selected_index + 1
        def on_prev(b):
            self.tab.selected_index = self.tab.selected_index - 1
        self.btn_next.on_click(on_next)
        self.btn_prev.on_click(on_prev)
        #######################
        self._main_screen = self.create_main_screen()
        self._strategy_found = self.create_strategy_found()
        self._train_choose = self.create_train_choose()
        self._manual_train_params = self.create_manual_params()
        self._retrain_widget = self.create_retrain_widget()
        self._log_widget = ipywidgets.Output(layout=ipywidgets.Layout(height='500px', border='solid',overflow_y='scroll'))
        def log_to_output(text, **kwargs):
            with self._log_widget:
                print(text, **kwargs)
        def err_to_output(text, **kwargs):
            with self._log_widget:
                print(text, **kwargs, file=sys.stderr)
        self.handlers["log"] = log_to_output
        self.handlers["error"] = err_to_output
        def set_state(state):
            self.state = state
        self.handlers["set_state"] = set_state
        #######################
        screen1 = ipywidgets.HBox([self._main_screen, self.btn_next])
        screen2 = ipywidgets.HBox([self._strategy_found, self.btn_next, self.btn_prev])
        screen3 = ipywidgets.HBox([self._manual_train_params, self.btn_next, self.btn_prev])
        screen4 = ipywidgets.HBox([self._train_choose, self.btn_next, self.btn_prev])
        screen5 = ipywidgets.HBox([self._retrain_widget, self.btn_prev])
        self.tab = ipywidgets.Tab()
        self.tab.children = [screen1, screen2, screen3, screen4, screen5]
        self.tab.titles = [str(i) for i in range(len(self.tab.children))]
        
        #######################
        clear_button = ipywidgets.Button(description='Очистить выдачу')
        def clear_log(b):
            self._log_widget.clear_output()
        clear_button.on_click(clear_log)
        
        self.window = ipywidgets.VBox([self.tab, self._log_widget,clear_button])
        for i in range(5):
            title = 'Screen' + str(i+1)
            self.tab.set_title(i, title)

    def display(self):
        display(self.window)
        #display(self.tab)
        #display(self._log_widget)

w = SolverGUI(None)
w.display()

In [8]:
w.state.task.fixedHyperParams

{'pipeline': 'ResNet18',
 'modelLastLayers': [{'Type': 'Dense', 'units': 64},
  {'Type': 'Activation', 'activation': 'relu'},
  {'Type': 'Dense', 'units': 1},
  {'Type': 'Activation', 'activation': 'sigmoid'}],
 'augmenParams': {'horizontal_flip': True,
  'preprocessing_function': 'keras.applications.resnet.preprocess_input',
  'vertical_flip': None,
  'width_shift_range': 0.4,
  'height_shift_range': 0.4},
 'loss': 'binary_crossentropy',
 'metrics': 'accuracy',
 'epochs': 150,
 'stoppingCriterion': 'stop_val_metr'}

In [6]:
w.proc._finished

True